# SEC 10-K RAG (Apple 2024 + Tesla 2023) — Colab Notebook

This notebook runs the full assignment pipeline:
- Upload PDFs
- Build FAISS index (embeddings)
- Retrieve top-5 + rerank
- Generate answers via **vLLM** (local/open LLM)
- Export `predictions.json`

**Tip:** Use a GPU runtime: *Runtime → Change runtime type → GPU*.


In [ ]:



GITHUB_REPO_URL = "https://github.com/<you>/<your-repo>.git"
!rm -rf sec_rag_vllm
!git clone {GITHUB_REPO_URL} sec_rag_vllm
%cd sec_rag_vllm


In [ ]:
# Install dependencies
!pip -q install -r requirements.txt
!pip -q install -r requirements-gpu-vllm.txt


## Upload the PDFs
Upload the two filings PDFs:
- Apple 2024 10-K PDF
- Tesla 2023 10-K PDF

They will be copied into `data/` as:
- `data/10-Q4-2024-As-Filed.pdf`
- `data/tsla-20231231-gen.pdf`


In [ ]:
from google.colab import files
import os, shutil
from pathlib import Path

Path('data').mkdir(exist_ok=True)
uploaded_pdfs = files.upload()

# Heuristic rename based on filename
apple_out = Path('data/10-Q4-2024-As-Filed.pdf')
tesla_out = Path('data/tsla-20231231-gen.pdf')

for fname in uploaded_pdfs.keys():
    if not fname.lower().endswith('.pdf'):
        continue
    lower = fname.lower()
    src = Path(fname)
    if 'tesla' in lower or 'tsla' in lower:
        shutil.copy(src, tesla_out)
        print('Saved Tesla PDF ->', tesla_out)
    elif 'apple' in lower or '10-q4-2024' in lower or 'q4-2024' in lower:
        shutil.copy(src, apple_out)
        print('Saved Apple PDF ->', apple_out)
    else:
        # If ambiguous, ask you to rename locally and re-upload; but we'll still keep it for manual move.
        dst = Path('data') / fname
        shutil.copy(src, dst)
        print('Saved unknown PDF ->', dst)

assert apple_out.exists(), 'Apple PDF not found in data/. Make sure you uploaded the Apple 10-K.'
assert tesla_out.exists(), 'Tesla PDF not found in data/. Make sure you uploaded the Tesla 10-K.'


## Configure the LLM
Set `RAG_LLM_MODEL` to an open-access instruct model supported by vLLM.

Good default for Colab GPUs: `microsoft/Phi-3-mini-4k-instruct`.


In [ ]:
import os

# Choose your model (open-access)
os.environ['RAG_LLM_MODEL'] = os.environ.get('RAG_LLM_MODEL', 'microsoft/Phi-3-mini-4k-instruct')

# Where the index will be stored
os.environ['RAG_INDEX_DIR'] = os.environ.get('RAG_INDEX_DIR', 'index')

print('RAG_LLM_MODEL =', os.environ['RAG_LLM_MODEL'])
print('RAG_INDEX_DIR =', os.environ['RAG_INDEX_DIR'])


## Build the index (PDF → chunks → embeddings → FAISS)


In [ ]:
!python -m rag_sec.cli ingest --data-dir data --index-dir index


## Run evaluation (13 questions) and export predictions


In [ ]:
!python -m rag_sec.cli eval --index-dir index --out outputs/predictions.json


In [ ]:
import json
from pathlib import Path

pred_path = Path('outputs/predictions.json')
print('Saved:', pred_path.resolve())
preds = json.loads(pred_path.read_text())
preds[:3]


## Interactive: call `answer_question(query)`


In [ ]:
from rag_sec import answer_question

answer_question('What was Apples total revenue for the fiscal year ended September 28, 2024?')


## Download the output


In [ ]:
from google.colab import files
files.download('outputs/predictions.json')
